<a href="https://colab.research.google.com/github/fatemafaria142/Analyzing-Large-Language-Models-with-LangChain/blob/main/Pdf_Summarization_using_LangChain_and_Mistral_7B_Instruct_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 

In [3]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 5.3 MB/s eta 0:00:00


In [4]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.7 MB/s eta 0:00:00


### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True)

* **Model and Tokenizer:** https://huggingface.co/openchat/openchat-3.5-1210

In [6]:
mode_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [7]:
model = AutoModelForCausalLM.from_pretrained(mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [41]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_k = 50  # You can adjust this value based on your preference
top_p = 0.8  # You can adjust this value based on your preference
temperature = 0.8  # You can adjust this value based on your preference

pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

### **Mount Drive**

In [30]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Import Necessary Libraries**
#### **Chunk Size:**

* **Definition:** Chunk size is the length of each segment or unit into which a text is divided.
* **Usage:** When processing long texts or documents, breaking them into smaller chunks can be more manageable for certain algorithms, models, or systems.
* **Example:** If you set chunk_size to 100, the text will be split into consecutive segments, each containing 100 characters.

#### **Chunk Overlap:**

* **Definition:** Chunk overlap is the number of characters shared between adjacent chunks.
* **Usage:** Overlapping chunks help capture context that may span across neighboring segments, ensuring that information is not lost at the boundaries of the chunks.
* **Example:** If you set chunk_overlap to 20 and chunk_size to 100, the first 80 characters of each chunk overlap with the last 80 characters of the preceding chunk.

#### **RecursiveCharacterTextSplitter**
* It takes a large text and splits it based on a specified chunk size. It does this by using a set of characters. The default characters provided to it are ["\n\n", "\n", " ", ""]. It takes in the large text then tries to split it by the first character \n\n. If the first split by \n\n is still large then it moves to the next character which is \n and tries to split by it. If it is still larger than our specified chunk size it moves to the next character in the set until we get a split that is less than our specified chunk size.

#### **CharacterTextSplitter**
* It takes the content of a document and splits it by the default separator(\n\n) which is the first level of chunking. If the first level of split creates a chunk greater than the specified chunk size, it does not split it further. However, if the first level of splitting generates smaller size chunk(less than the specified chunk size), it attempts to merge it with another chunk to adhere to the specified chunk size. So, if the first level split generates chunks greater than the chunk size, this logic will be not be of much use.

In [31]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

In [32]:
# Initializes a DirectoryLoader to load text files from the specified directory.
'''
path = "/content/drive/MyDrive/pdf_examples"
loader = DirectoryLoader(path, glob="./*.pdf", loader_cls=PyPDFLoader)
# Loads all the documents present in the directory.
documents = loader.load()
'''
# load document
loader = PyPDFLoader("/content/drive/MyDrive/pdf_examples/CV_of_Fatema_Tuj_Johora_Faria.pdf")
documents = loader.load()

In [33]:
print(documents)

[Document(page_content='Fatema Tuj Johora Faria\n/ne+8801768158539\nTitas Jahanara Monjil, 323 East Goran\nfatema.faria142@gmail.com\n/♀nednFatema Tuj Johora Faria\n/gtbhttps://github.com/fatemafaria142\nEducation\nAhsanullah University of Science and Technology Dhaka, Bangladesh\nB. Sc. in Computer Science and Engineering Spring 2019 - Fall 2023\nCGPA: 3.302 on a scale of 4.00\nNational Ideal College Dhaka, Bangladesh\nHigher Secondary School Certificate (HSC) 2015 - 2017\nGPA: 5.00 on a scale of 5.00\nMotijheel Model School & College Dhaka, Bangladesh\nSecondary School Certificate (SSC) 2013 - 2015\nGPA: 5.00 on a scale of 5.00\nUndergraduate Thesis\nGenerative Adversarial Networks for Crop Disease: A Case Study with Potato Disease Classifi-\ncation and Instance Segmentation\nSupervisor: Dr. Mohammad Shafiul Alam, Professor, Department of CSE, AUST\nTechnical Skills\n◦Programming Language: Python (NumPy, SciPy, Matplotlib, Pandas, Seaborn), C++\n◦Web Development: HTML5, CSS3, JavaScr

In [35]:
# Initialize a RecursiveCharacterTextSplitter with a specified chunk size and overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

# Split the loaded documents into chunks of text using the defined text_splitter.
docs = text_splitter.split_documents(documents)

# Check the length of the resulting text chunks.
print("Number of text chunks:", len(docs))


Number of text chunks: 12


### **Some Examples**

In [36]:
for i in range(3):  # Adjust the range based on the number of chunks you want to print
    print(f"{i + 1}st chunk: {docs[i].page_content}\n")


1st chunk: Fatema Tuj Johora Faria
/ne+8801768158539
Titas Jahanara Monjil, 323 East Goran
fatema.faria142@gmail.com
/♀nednFatema Tuj Johora Faria
/gtbhttps://github.com/fatemafaria142
Education
Ahsanullah University of Science and Technology Dhaka, Bangladesh
B. Sc. in Computer Science and Engineering Spring 2019 - Fall 2023
CGPA: 3.302 on a scale of 4.00
National Ideal College Dhaka, Bangladesh
Higher Secondary School Certificate (HSC) 2015 - 2017
GPA: 5.00 on a scale of 5.00

2st chunk: Motijheel Model School & College Dhaka, Bangladesh
Secondary School Certificate (SSC) 2013 - 2015
GPA: 5.00 on a scale of 5.00
Undergraduate Thesis
Generative Adversarial Networks for Crop Disease: A Case Study with Potato Disease Classifi-
cation and Instance Segmentation
Supervisor: Dr. Mohammad Shafiul Alam, Professor, Department of CSE, AUST
Technical Skills
◦Programming Language: Python (NumPy, SciPy, Matplotlib, Pandas, Seaborn), C++

3st chunk: ◦Web Development: HTML5, CSS3, JavaScript, Django

### **Documentation Link:** https://python.langchain.com/docs/use_cases/summarization
### **Option 1. Stuff**
* When we use `load_summarize_chain` with `chain_type="stuff"`, we will use the StuffDocumentsChain.

* The chain will take a list of documents, inserts them all into a prompt, and passes that prompt to an LLM:

In [38]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

# Define prompt
prompt_template = """[INST] Envision yourself as a seasoned CV summarizer, known for your expertise in extracting key information. A proactive student reaches out to you, eager to receive your valuable guidance on optimizing their CV. Your objective is to provide a thorough and succinct summary of the student's CV, ensuring clarity and accuracy. Delve into the details of their achievements, skills, and experiences, distilling the essence of their professional profile in your response. [/INST]"

"{text}"

CONCISE SUMMARY:
"""

prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")
docs = docs[:5]
print(stuff_chain.run(docs))


Fatema Tuj Johora Faria is a Computer Science Engineering graduate from Ahsanullah University of Science and Technology, Bangladesh, with a CGPA of 3.302. She holds a Higher Secondary School Certificate from National Ideal College, Bangladesh, and a Secondary School Certificate from Motijheel Model School & College, Bangladesh, both with a perfect GPA of 5.00.

Fatema has a strong foundation in programming languages such as Python and C++, web development with HTML5, CSS3, and JavaScript, and database management using MySQL, MongoDB, and SQLAlchemy. She has expertise in business intelligence tools like Tableau, IDEs such as Visual Studio Code, PyCharm, and Anaconda Navigator, deep learning frameworks like TensorFlow, Keras, PyTorch, and Scikit-learn, and cloud computing on AWS.

Fatema's projects showcase her proficiency in SQL query generation using large language models, mental health advice generation, text-to-code generation pipeline, math word problem solving, and agronomic quest